#### SQLite port_lite database: sales, stocks table
#### csv files: daily-watch-list, daily-sales, monitors, stocks-all

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 7, 3)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-07-03
yesterday: 2023-06-30


### Restart and Run All Cells | daily-watch-list & daily-sales upload to obsidian

In [3]:
#cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','chg_acc':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','%chg_acc':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1276404,WHART,2023-06-29,2023-06-30,1,10.60,10.60,0,0.00%,0,0,10.70,10.50,"2,839,723",2023-07-02,2023-07-02,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.80,I,11.00,11.00,24.74,0.21,11.70,"10,000",-5,4,"30,000",0,0,5pct,SET


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-06-30'
ORDER BY a.name



,name
0,AH
1,AWC
2,BBL
3,BCH
4,BDMS
5,BEM
6,BH
7,CPNREIT
8,DIF
9,EA


In [9]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-06-30' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [10]:
df = pd.read_sql(sql, conlite)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('chg_acc = to_price - fm_price',inplace=True)
df['%chg_acc'] = round(df['chg_acc']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,chg_acc,%chg_acc
0,CPNREIT,2023-06-28,2023-06-30,10.80,10.80,"14,754,039",11.40,10.50,B,SET,0.00,0.00%
1,DIF,2023-06-27,2023-06-30,10.70,11.10,"86,464,990",11.20,10.60,B,SET,0.40,3.74%
2,GVREIT,2023-06-30,2023-06-30,8.35,8.35,"88,700",8.50,8.30,B,SET,0.00,0.00%
3,JMART,2023-06-29,2023-06-30,15.60,15.90,"118,560,329",16.40,13.60,B,SET50,0.30,1.92%
4,JMT,2023-06-29,2023-06-30,36.00,37.50,"43,122,989",37.50,34.00,B,SET50,1.50,4.17%
5,MCS,2023-06-30,2023-06-30,6.35,6.35,"467,759",6.45,6.20,B,SET,0.00,0.00%
6,SENA,2023-06-29,2023-06-30,2.96,2.98,"1,880,060",3.02,2.90,B,SET,0.02,0.68%
7,SINGER,2023-06-30,2023-06-30,9.00,9.00,"9,471,895",9.40,8.95,B,SET100,0.00,0.00%
8,STA,2023-06-29,2023-06-30,17.00,17.00,"11,230,604",17.60,16.70,B,SET100,0.00,0.00%
9,SYNEX,2023-06-28,2023-06-30,11.90,11.90,"1,217,951",12.40,11.80,B,SET,0.00,0.00%


### Price change up or down more than 3%

In [11]:
# For each row in the DataFrame, generate a series of dates between the start and end dates
df['dates'] = df.apply(lambda x: pd.date_range(x['fm_date'], x['to_date']), axis=1)

# For each row in the DataFrame, select only the business days (Monday through Friday) from the series of dates
df['business_days'] = df['dates'].apply(lambda x: x[x.isin(pd.date_range(x.min(), x.max(), freq='B'))])

# For each row in the DataFrame, count the number of business days between the start and end dates
df['bdays'] = df['business_days'].apply(lambda x: len(x))

In [12]:
colt =  'name %chg_acc status fm_price to_price market \
fm_date bdays max_price min_price'.split()
watch_list = df.loc[abs(df['%chg_acc']) >= 3,colt]
watch_list.sort_values('%chg_acc',ascending=False)
watch_list

,name,%chg_acc,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
1,DIF,3.74,B,10.70,11.10,SET,2023-06-27,4,11.20,10.60
4,JMT,4.17,B,36.00,37.50,SET50,2023-06-29,2,37.50,34.00
11,AH,7.75,I,32.25,34.75,SET,2023-06-27,4,34.75,30.75
12,AWC,3.35,I,4.78,4.94,SET50,2023-06-28,3,4.98,4.76
14,ICHI,5.34,I,13.10,13.80,SET,2023-06-28,3,14.00,12.80
26,ORI,4.95,O,10.10,10.60,SET100,2023-06-29,2,10.60,10.20


In [13]:
# cannot use with column that starts with %
#df.query('abs(%chg_acc) >= 1.0')

In [14]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(output_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(data_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(data_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(one_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(osd_file, index=False)

### Create daily-sales from sales

In [15]:
df[colt].style.format(format_dict)

,name,%chg_acc,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
0,CPNREIT,0.00%,B,10.80,10.80,SET,2023-06-28,3,11.40,10.50
1,DIF,3.74%,B,10.70,11.10,SET,2023-06-27,4,11.20,10.60
2,GVREIT,0.00%,B,8.35,8.35,SET,2023-06-30,1,8.50,8.30
3,JMART,1.92%,B,15.60,15.90,SET50,2023-06-29,2,16.40,13.60
4,JMT,4.17%,B,36.00,37.50,SET50,2023-06-29,2,37.50,34.00
5,MCS,0.00%,B,6.35,6.35,SET,2023-06-30,1,6.45,6.20
6,SENA,0.68%,B,2.96,2.98,SET,2023-06-29,2,3.02,2.90
7,SINGER,0.00%,B,9.00,9.00,SET100,2023-06-30,1,9.40,8.95
8,STA,0.00%,B,17.00,17.00,SET100,2023-06-29,2,17.60,16.70
9,SYNEX,0.00%,B,11.90,11.90,SET,2023-06-28,3,12.40,11.80


In [16]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(output_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(data_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(one_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(osd_file, index=False)

### Create monitors from orders

In [17]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.sample(5)

,trade
name,
BH,B
ICHI,S
JASIF,S
KTB,B
BCH,S


In [18]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,26
1,S,9


### output_file = iCloud Drive on PC

In [19]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(output_file)
monitors.to_csv(one_file)

### End of daily process

In [20]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,26
1,S,9


In [21]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
CPNREIT,B,SET
DIF,B,SET
GVREIT,B,SET
JMART,B,SET50
JMT,B,SET50
...,...,...
TCAP,X,SET100
TFG,X,SET
TOP,X,SET50


In [22]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(one_file)

In [23]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
BBL,B
BDMS,B
BEM,B
BH,B
CPNREIT,B
DIF,B
EA,B
GVREIT,B
JMART,B


In [24]:
buy_candidates.shape[0]

26

In [25]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
AH,S
AWC,S
BCH,S
ICHI,S
IVL,S
JASIF,S
KTC,S
PTTGC,S
WHART,S


In [26]:
sell_candidates.shape[0]

9